In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
from datetime import date
from datetime import datetime
import numpy as np

from summer.utils import ref_times_to_dti

from autumn.tools.inputs.demography.queries import get_population_by_agegroup
from autumn.models.covid_19.detection import create_cdr_function
from autumn.tools.utils.utils import apply_moving_average
from autumn.tools.curve.scale_up import scale_up_function
from autumn.tools.project import get_project
from autumn.settings import Region, Models
from autumn.models.covid_19.constants import AGEGROUP_STRATA
from autumn.models.covid_19.mixing_matrix.macrodistancing import weight_mobility_data
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.tools import inputs
from autumn.tools.inputs.database import get_input_db
from autumn.tools.utils.display import pretty_print
from autumn.tools.inputs.social_mixing.build_synthetic_matrices import build_synthetic_matrices
from autumn.models.covid_19.detection import get_testing_numbers_for_region

In [ ]:
age_integers = [int(group) for group in AGEGROUP_STRATA]
model = Models.SM_SIR
region = Region.BHUTAN
project = get_project(model, region, reload=True)
base_params = project.param_set.baseline

## Population

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
age_groups = base_params["age_groups"]

total_pops = inputs.get_population_by_agegroup(
    age_groups, 
    base_params["country"]["iso3"],
    None,
#     "FDMN", 
    year=base_params["population"]["year"]
)
print(f"total modelled population of {region} is: {round(sum(total_pops) / 1e3, 3)} thousand")
ax.bar(age_groups, total_pops, width=4)
ax.set_title(region)
ax.set_ylabel("population")
ax.set_xlabel("starting age of age bracket")
fig.suptitle("population distribution by age")

## Mixing matrix
### Check how mixing matrix is specified for each region

In [ ]:
print(f"Modelled country: {project.param_set.baseline['country']['iso3']}")
print(f"Modelled sub-region: {project.param_set.baseline['population']['region']}")
print(f"Proxy country: {project.param_set.baseline['ref_mixing_iso3']}")
print("Always age-adjusted under SM-SIR code")

### Display the matrix and the matrix components

In [ ]:
agegroup_types = {
    "base age groups": AGEGROUP_STRATA,
    "modelled age groups": project.param_set.baseline["age_groups"],
}

for title, agegroups in agegroup_types.items():

    mixing_matrix = build_synthetic_matrices(
        project.param_set.baseline["country"]["iso3"],
        project.param_set.baseline["ref_mixing_iso3"],
        agegroups,
        True,
        project.param_set.baseline["population"]["region"]
    )

    fig = plt.figure(figsize=(12, 8))
    positions = [1, 2, 3, 5, 6]
    for i_loc, location in zip(positions, mixing_matrix.keys()):
        ax = fig.add_subplot(2, 3, i_loc)
        ax.imshow(
            np.flipud(np.transpose(mixing_matrix[location])), 
            cmap=cm.hot, 
            vmin=0,
            vmax=mixing_matrix[location].max(), 
            origin="lower"
        )
        ax.set_title(location.replace("_", " "))
        ax.set_xticks([])
        ax.set_yticks([])
    fig.suptitle(title)

## Case detection

In [ ]:
testing_params = project.param_set.baseline["testing_to_detection"]
print("Parameter values are: ")
pretty_print(testing_params)

In [ ]:
# Get the CDR function of tests
cdr_from_tests_func = create_cdr_function(
    testing_params["assumed_tests_parameter"],
    testing_params["assumed_cdr_parameter"],
)

# Get the denominator population
testing_pops = get_population_by_agegroup(
    project.param_set.baseline["age_groups"],
    project.param_set.baseline["country"]["iso3"],
    project.param_set.baseline["population"]["region"]
)

# Process the data
test_times, test_values = get_testing_numbers_for_region("BTN", None)
test_dates = ref_times_to_dti(COVID_BASE_DATETIME, [int(time) for time in test_times])
per_capita_tests = [i_tests / sum(testing_pops) for i_tests in test_values]
dummy_tests = np.linspace(0, max(per_capita_tests), 200)
if testing_params["assumed_tests_parameter"]:
    smoothed_per_capita_tests = apply_moving_average(
        per_capita_tests, 
        testing_params["smoothing_period"]
    )
else:
    smoothed_per_capita_tests = per_capita_tests
cdr_function_of_time = scale_up_function(
    test_times,
    [cdr_from_tests_func(test_rate) for test_rate in smoothed_per_capita_tests],
    smoothness=0.2, method=4, bound_low=0.,
)    

# Plot
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.tight_layout(w_pad=1.5, h_pad=5)

def sort_axis_dates(ax):
    axis.tick_params(axis="x", labelrotation=45)
    axis.set_xlim(left=plot_left_date, right=plot_right_date)

# Plot daily number of tests
axis = axes[0, 0]
axis.plot(test_dates, test_values, marker="o")
axis.set_title("daily testing numbers")
sort_axis_dates(axis)

# Plot daily number of tests
axis = axes[0, 1]
axis.plot(test_dates, per_capita_tests, label="raw")
axis.plot(test_dates, smoothed_per_capita_tests, label="smoothed")
axis.set_title("daily per capita testing rate")
sort_axis_dates(axis)
axis.legend()

# Plot relationship of daily tests to CDR proportion
axis = axes[1, 0]
axis.plot(dummy_tests, cdr_from_tests_func(dummy_tests))
axis.scatter(per_capita_tests, [cdr_from_tests_func(i_tests) for i_tests in per_capita_tests], color="r")
axis.set_ylabel("case detection proportion")
axis.set_xlabel("per capita testing rate")
axis.set_title("daily per capita tests to CDR relationship")
axis.set_ylim(top=1.)

# Plot CDR values
axis = axes[1, 1]
multiplier = 1e2
axis.scatter(test_dates, [cdr_from_tests_func(i_test_rate) * multiplier for i_test_rate in smoothed_per_capita_tests], color="r")
axis.plot(test_dates, [cdr_function_of_time(time) * multiplier for time in test_times])
axis.set_title("Final case detection rate")
axis.set_ylabel("percentage")
sort_axis_dates(axis)

fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 10))
axis = axes[0]
axis.plot(test_dates, test_values, marker=".", linewidth=0)
axis.set_title("daily number of tests done")
axis = axes[1]
axis.plot(test_dates, per_capita_tests, marker=".", linewidth=0)
axis.set_title("daily per capita testing rate")
fig.tight_layout()